In [7]:
# python script of M4_weekly and DeepAR

# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator


# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

if __name__ == "__main__":

    import mxnet as mx
    from pprint import pprint

    dataset = get_dataset("m4_daily", regenerate=False)

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=50,      # default: 100
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
        # hybridize=False,
    )

    # cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        # cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )

    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_daily.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 13463
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 13.43it/s, avg_epoch_loss=7.9]
INFO:root:Epoch[0] Elapsed time 3.728 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.897492
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.14it/s, avg_epoch_loss=6.57]
INFO:root:Epoch[1] Elapsed time 4.126 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.568923
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 11.27it/s, avg_epoch_loss=6.61]
INFO:root:Epoch[2] Elapsed time 4.441 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.608401
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.15it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[3] Elapsed time 4.126 seconds
INFO:

INFO:root:Epoch[36] Elapsed time 7.043 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.004645
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:07<00:00,  6.40it/s, avg_epoch_loss=5.87]
INFO:root:Epoch[37] Elapsed time 7.815 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.868467
INFO:root:Loading parameters from best epoch (27)
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 50/50 [00:08<00:00,  6.12it/s, avg_epoch_loss=5.83]
INFO:root:Epoch[38] Elapsed time 8.187 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=5.825631
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 50/50 [00:08<00:00,  6.09it/s, avg_epoch_loss=5.93]
INFO:root:Epoch[39] Elapsed time 8.215 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=5.934922
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 50/50 [00:07<00:00,  6.66it/s, avg_epoch_loss=5.74]
INFO:root:Epoch[40] Elapsed time 7.512 seconds
INFO:root:Epoch[40] Eval

{'Coverage[0.1]': 0.4679948629558289,
 'Coverage[0.2]': 0.6214640575889679,
 'Coverage[0.3]': 0.7176146540944284,
 'Coverage[0.4]': 0.7884855858596108,
 'Coverage[0.5]': 0.8386224610497148,
 'Coverage[0.6]': 0.8752407989455535,
 'Coverage[0.7]': 0.9078542701679672,
 'Coverage[0.8]': 0.9358376423670948,
 'Coverage[0.9]': 0.9610159180776633,
 'MAE_Coverage': 0.29045891678964775,
 'MASE': 5.391340205717229,
 'MSE': 506260.88738305744,
 'MSIS': 95.55869083570394,
 'ND': 0.04192610811548718,
 'NRMSE': 0.10991019530767762,
 'QuantileLoss[0.1]': 12907376.523580931,
 'QuantileLoss[0.2]': 15695639.471469117,
 'QuantileLoss[0.3]': 16785939.829965208,
 'QuantileLoss[0.4]': 16824243.19220581,
 'QuantileLoss[0.5]': 16061793.668121338,
 'QuantileLoss[0.6]': 14487287.949664306,
 'QuantileLoss[0.7]': 12410583.071714785,
 'QuantileLoss[0.8]': 9661291.525701903,
 'QuantileLoss[0.9]': 6098779.529747008,
 'RMSE': 711.5201243696888,
 'abs_error': 16061793.640136719,
 'abs_target_mean': 6473.649895515986,
 